# Why Does It Churn, Smeagol?

## Overview

(Client Redacted) approached our company, The Classification Station, to ascertain whether we could build a model to accurately predict whether a customer would "soon" stop doing business with (Client Redacted). When a customer withdraws their business, this is known as "churning".

## Business Understanding

While a certain amount of churn is unavoidable, businesses strive to bring churn to as low a level as possible. In essence, they'd like to keep their customers if they can. So, our company was employed to determined causes of churn and which of those causes had the most impact on (Client Redacted). Specifically, as we begin 2024, it is wise to keep in mind the changing social landscape around us. I do not know a single person without a cell phone unless they are a very young child, but I know many people without a landline, or only a cursory one that they got along with their internet or cable television package. Therefore, this project will continue under the assumption we are speaking of a cellular telephone company rather than one offering landline services.

## Data Understanding

This public dataset is provided by the CrowdAnalytix community as part of their churn prediction competition. The real name of the telecom company is anonymized. It contains 20 predictor variables mostly about customer usage patterns. There are 3333 records in this dataset, out of which 483 customers are churners and the remaining 2850 are non-churners. Thus, the ratio of churners in this dataset is 14%. Our first real steps at understanding our data, outside of the metadata we have, is performing a .head(), .describe() and .info() to see some basic statistics about our data.

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline as ImbPipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from imblearn.over_sampling import SMOTE

Our first step is to pull in our data and do give it a cursory look-through, simply to see what we're working with.

In [3]:
cust_df = pd.read_csv("Data/bigml_59c28831336c6604c800002a.csv")

In [4]:
cust_df.head()

,state,account length,area code,phone number,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,...,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [5]:
cust_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   state                   3333 non-null   object 
 1   account length          3333 non-null   int64  
 2   area code               3333 non-null   int64  
 3   phone number            3333 non-null   object 
 4   international plan      3333 non-null   object 
 5   voice mail plan         3333 non-null   object 
 6   number vmail messages   3333 non-null   int64  
 7   total day minutes       3333 non-null   float64
 8   total day calls         3333 non-null   int64  
 9   total day charge        3333 non-null   float64
 10  total eve minutes       3333 non-null   float64
 11  total eve calls         3333 non-null   int64  
 12  total eve charge        3333 non-null   float64
 13  total night minutes     3333 non-null   float64
 14  total night calls       3333 non-null   

Some very basic data exploration tells us that our dataframe contains information about customers, such as their state, length of account time, phone number, total minutes and charges, customer service calls, and whether they "churned". Recall that "churning" is to stop doing business with the company, so in this case, that column tells us whether the customer still does business with (Client Redacted). These explorations also tell us that we have 21 columns with 3333 entries each, and no null values. Furthermore, our data types include objects, integers, and one Boolean column.

One important thing to note was discussed earlier - in this dataset, only 14% of customers have "churned". We can see that here -

In [6]:
cust_df['churn'].value_counts(normalize=True)

False    0.855086
True     0.144914
Name: churn, dtype: float64

This means that whilst we do have a binary classification problem on our hands, very suitable for logistic regression, we also have a class imbalance. Obviously, we will not be able to accurately predict anything with only 14% of a category of data available to us. We will deal with this using a technique called SMOTE which allows us to, at our discretion, oversample the minority or undersample the majority, or both. It is not recommended to undersample the majority as it amounts to leaving data on the table, and so we will be oversampling the minority in this case. However, we will revisit this topic, as I intend to use SMOTE within a pipeline.

Here, we determine which of these numeric columns have a correlation with our "churn" column, as that will eventually be our target when it comes time to model. A correlation of +/- 0.06 is considered notable, and as stated, we care about which columns are most strongly correlated (positively or negatively) with our "churn" column.

In [7]:
cust_df.corr()

,account length,area code,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn
account length,1.000000,-0.012463,-0.004628,0.006216,0.038470,0.006214,-0.006757,0.019260,-0.006745,-0.008955,-0.013176,-0.008960,0.009514,0.020661,0.009546,-0.003796,0.016541
area code,-0.012463,1.000000,-0.001994,-0.008264,-0.009646,-0.008264,0.003580,-0.011886,0.003607,-0.005825,0.016522,-0.005845,-0.018288,-0.024179,-0.018395,0.027572,0.006174
number vmail messages,-0.004628,-0.001994,1.000000,0.000778,-0.009548,0.000776,0.017562,-0.005864,0.017578,0.007681,0.007123,0.007663,0.002856,0.013957,0.002884,-0.013263,-0.089728
total day minutes,0.006216,-0.008264,0.000778,1.000000,0.006750,1.000000,0.007043,0.015769,0.007029,0.004323,0.022972,0.004300,-0.010155,0.008033,-0.010092,-0.013423,0.205151
total day calls,0.038470,-0.009646,-0.009548,0.006750,1.000000,0.006753,-0.021451,0.006462,-0.021449,0.022938,-0.019557,0.022927,0.021565,0.004574,0.021666,-0.018942,0.018459
total day charge,0.006214,-0.008264,0.000776,1.000000,0.006753,1.000000,0.007050,0.015769,0.007036,0.004324,0.022972,0.004301,-0.010157,0.008032,-0.010094,-0.013427,0.205151
total eve minutes,-0.006757,0.003580,0.017562,0.007043,-0.021451,0.007050,1.000000,-0.011430,1.000000,-0.012584,0.007586,-0.012593,-0.011035,0.002541,-0.011067,-0.012985,0.092796
total eve calls,0.019260,-0.011886,-0.005864,0.015769,0.006462,0.015769,-0.011430,1.000000,-0.011423,-0.002093,0.007710,-0.002056,0.008703,0.017434,0.008674,0.002423,0.009233
total eve charge,-0.006745,0.003607,0.017578,0.007029,-0.021449,0.007036,1.000000,-0.011423,1.000000,-0.012592,0.007596,-0.012601,-0.011043,0.002541,-0.011074,-0.012987,0.092786
total night minutes,-0.008955,-0.005825,0.007681,0.004323,0.022938,0.004324,-0.012584,-0.002093,-0.012592,1.000000,0.011204,0.999999,-0.015207,-0.012353,-0.015180,-0.009288,0.035493


One unfortunate caveat to this method of ascertaining correlation is that categorical columns, such as "state", will be ignored. This is because the .corr() function can only make use of numerical data. Still, it is a start, and our catagorical data correlation will be dealt with later as we establish a pipeline to do just that! In any case, this does tell us which of our numeric columns might be most useful to us as we begin to build our model (and our pipeline).

All that remains in our first exploratory steps within the wonderful world of our customer data is to make an action plan and perform a few last steps to enhance our understanding as we move into the preparation phase. As stated, I will be employing pipelines both to clean and prepare my training and testing data and to tune and evaluate the models fitted to that draining data. Therefore, I will discuss SMOTE, scaling, and encoding when I build the pipeline to perform these actions.

The very final step could also be performed in a pipeline, but I have chosen to do it manually for several reasons. First, dropping in a pipeline can cause trouble for OneHotEncoder, which I am planning to use. So in the interest of using my noodle ahead of time, I am simply going to perform this manually. As a selfish little aside, I also want to display that I've learned how to drop columns better since phase one thank you.

In [8]:
cust_df = cust_df.drop(['area code', 'phone number', 'international plan', 'voice mail plan', 'total day calls', 'total eve calls', 'total night calls', 'total intl calls'], axis=1)

In [9]:
cust_df

,state,account length,number vmail messages,total day minutes,total day charge,total eve minutes,total eve charge,total night minutes,total night charge,total intl minutes,total intl charge,customer service calls,churn
0,KS,128,25,265.1,45.07,197.4,16.78,244.7,11.01,10.0,2.70,1,False
1,OH,107,26,161.6,27.47,195.5,16.62,254.4,11.45,13.7,3.70,1,False
2,NJ,137,0,243.4,41.38,121.2,10.30,162.6,7.32,12.2,3.29,0,False
3,OH,84,0,299.4,50.90,61.9,5.26,196.9,8.86,6.6,1.78,2,False
4,OK,75,0,166.7,28.34,148.3,12.61,186.9,8.41,10.1,2.73,3,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3328,AZ,192,36,156.2,26.55,215.5,18.32,279.1,12.56,9.9,2.67,2,False
3329,WV,68,0,231.1,39.29,153.4,13.04,191.3,8.61,9.6,2.59,3,False
3330,RI,28,0,180.8,30.74,288.8,24.55,191.9,8.64,14.1,3.81,2,False
3331,CT,184,0,213.8,36.35,159.6,13.57,139.2,6.26,5.0,1.35,2,False


As we can see, we've dropped several columns I feel are irrelevant. I don't feel that a person's area code or phone number would influence their decision to leave a cell phone provider, specifically. As a matter of fact, one of the most famous phone numbers of all time (867-5309) was last sold in 2009 for upwards of 365,000. Customers are far more likely to request a simple (and usually free with good reason) phone number change rather than churn.

Additionally, according to the available data, voice mail plans do not cost the customer any extra, and so I do not believe the correlation would be high enough to keep it in as a later feature for our model. I also included the number of voice mail messages, which I feel would get the same point across whether the customer had a voice mail plan or not. Similarly, I felt that all of the data contained within number of calls dependent on time of day or international was also contained within the minutes columns, and so chose to keep those instead.

## Data Preparation

Since we are modeling (and testing that model), our first step will always be to complete a train/test split. Data leakage is a very serious concern in machine learning, because our models are trained on a set of data and then tested on another set that is supposed to have remained unseen during the training phase. Data leakage occurs when that is not the case, and the model has been "trained" on "testing" data. This is disingenuous and skews results, so it is of the utmost importance that we prevent said data leakage.

In [193]:
X = cust_df.drop('churn', axis=1)
y = cust_df['churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=42)

In [194]:
X_train

,state,account length,international plan,number vmail messages,total day minutes,total day charge,total eve minutes,total eve charge,total night minutes,total night charge,total intl minutes,total intl charge,customer service calls
367,MD,45,no,0,78.2,13.29,253.4,21.54,255.0,11.48,18.0,4.86,1
3103,DE,115,no,0,195.9,33.30,227.0,19.30,313.2,14.09,13.2,3.56,2
549,OK,121,no,31,237.1,40.31,205.6,17.48,196.7,8.85,10.1,2.73,4
2531,RI,180,no,0,143.3,24.36,180.5,15.34,184.2,8.29,10.1,2.73,1
2378,OR,112,no,0,206.2,35.05,164.5,13.98,140.3,6.31,12.6,3.40,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,ID,106,no,0,274.4,46.65,198.6,16.88,160.8,7.24,6.0,1.62,1
1130,PA,122,no,0,35.1,5.97,180.8,15.37,251.6,11.32,12.7,3.43,1
1294,OR,66,no,0,87.6,14.89,262.0,22.27,184.6,8.31,9.2,2.48,1
860,HI,169,no,0,179.2,30.46,175.2,14.89,228.6,10.29,9.9,2.67,2


In [195]:
X_test

,state,account length,international plan,number vmail messages,total day minutes,total day charge,total eve minutes,total eve charge,total night minutes,total night charge,total intl minutes,total intl charge,customer service calls
438,WY,113,no,0,155.0,26.35,330.6,28.10,189.4,8.52,13.5,3.65,1
2674,IL,67,no,0,109.1,18.55,217.4,18.48,188.4,8.48,12.8,3.46,0
1345,SD,98,no,0,0.0,0.00,159.6,13.57,167.1,7.52,6.8,1.84,4
1957,KY,147,no,0,212.8,36.18,204.1,17.35,156.2,7.03,10.2,2.75,1
2148,WY,96,no,0,144.0,24.48,224.7,19.10,227.7,10.25,10.0,2.70,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3257,NY,171,no,0,137.5,23.38,198.1,16.84,292.7,13.17,13.3,3.59,2
1586,CT,89,no,0,82.3,13.99,167.2,14.21,194.7,8.76,7.2,1.94,1
3068,SC,78,no,21,160.6,27.30,223.1,18.96,124.0,5.58,9.5,2.57,2
2484,IN,141,no,39,116.9,19.87,276.5,23.50,289.9,13.05,12.3,3.32,0


As we can see, our data has now been split into a training and a testing set. The testing set will be set aside and will not be touched until it is time to test the final model on that testing data. When that time comes, I will use the data cleaning pipeline I am about to build in order to SMOTE, scale, and OneHotEncode the testing data in the same way it will do to the training data. Technically, the train/test split step could have been performed in the cleaning pipeline as well, but I preferred to do it myself and ensure that the testing data was properly set aside and untouched throughout the process.

Finally, with all these beginning steps completed, we can build a pipeline to get to the true meat of our data preparation.

In [196]:
num_feats = ['account length', 'number vmail messages', 'total day minutes', 'total day charge', 'total eve minutes', 'total eve charge', 'total night minutes', 'total night charge', 'total intl minutes', 'total intl charge', 'customer service calls']
cat_feats = ['state', 'international plan']

In [211]:
standard_pipeline = ColumnTransformer(
    transformers=[
        ('categorical', OneHotEncoder(drop='first'), cat_feats),
        ('numeric', StandardScaler(), num_feats)
    ],
    remainder='passthrough')

In [212]:
standard_pipeline.fit_transform(X_train)

<2499x62 sparse matrix of type '<class 'numpy.float64'>'
	with 30192 stored elements in Compressed Sparse Row format>

## Analysis

## Model Recommendation

## Conclusion

## Next Steps